# Szenario  

In diesem Szenario sind wir ein Data Analyst bei einem high-tech Hersteller für gesundheitsbezogene Produkte für Frauen. Die Chief Creative Officer Urška Sršen glaubt, dass die Analyse des Kundenverhaltens neue Wachstumschancen aufzeigen kann. 

Es gilt dabei folgende Fragen zu beantworten. 
1. Was sind Trends in den Daten? 
2. Wie könnten diese Trends einen Einfluss auf die Marketingstrategie haben? 




# Laden der benötigten Pakete

In [ ]:
library(tidyverse)

# Laden der Daten
Wir laden die verschiedenen csv-Dateien in Data Frames ein. 
Die verfügbaren Daten sind aufgeteilt in verschiedene Genauigkeiten. Für diese Analyse reichen die täglichen Daten aus. 

In [ ]:
daily_activity <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")
daily_calories <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyCalories_merged.csv")
daily_intensities <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyIntensities_merged.csv")
daily_steps <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailySteps_merged.csv")
heartrate_seconds <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv")
sleep_day <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv")
weight_info <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/weightLogInfo_merged.csv")

# Data-Cleaning
Wir werden Duplikate, Null-Werte, die allgemeine Logik und Datentypen überprüfen.

## Duplikate

In [ ]:
# duplicate daily_activity check
sum(duplicated(daily_activity))

# duplicate daily_calories check
sum(duplicated(daily_calories))

# duplicate daily_intensities check
sum(duplicated(daily_intensities))

# duplicate daily_steps check
sum(duplicated(daily_steps))

# duplicate heartrate_seconds check
sum(duplicated(heartrate_seconds))

# duplicate sleep_day check
sum(duplicated(sleep_day))

# duplicate weight_info check
sum(duplicated(weight_info))

Es zeigt sich, dass sleep_day Duplikate enthält. Wir wollen diese Duplikate entfernen. 

In [ ]:
# Duplikate entfernen
sleep_day <- sleep_day %>%
  distinct()
# Überprüfung
sum(duplicated(sleep_day))

Es befinden sich keine Duplikate mehr in den Dataframes. 

## Null-Werte
Wie überprüfen alle Dataframes auf Null-Werte

In [ ]:
which(is.na(daily_activity), arr.ind=TRUE)
which(is.na(daily_calories), arr.ind=TRUE)
which(is.na(daily_intensities), arr.ind=TRUE)
which(is.na(daily_steps), arr.ind=TRUE)
which(is.na(heartrate_seconds), arr.ind=TRUE)
which(is.na(sleep_day), arr.ind=TRUE)
which(is.na(weight_info), arr.ind=TRUE)

Es zeigt sich, dass weight_info viele Null-Stellen in Spalte 5 enthält. Wir wollen wissen, was Spalte 5 ist. 

In [ ]:
head(weight_info %>% select(5))

Spalte 5 ist die Fettmessung. Es ergibt Sinn, dass nicht jede Gewichtsmessung auch eine Fett-Messung beinhaltet, da eine Fettmessung spezielles Equipment erfordert. Dieses Equipment hat nicht jeder zu Hause.  
Es sind entsprechend keine Null-Werte enthalten, die bereinigt werden müssten. 

## Logik der Daten
Beim Sichten der Tabellen in Views ist aufgefallen, dass es einige Tagesmessungen mit 0 Schritten oder 1440 Minuten (= 24 Stunden) Inaktivität gibt. 
Wir entfernen die Messungen mit 0 Schritten am Tag und völliger Inaktivität, da davon ausgegangen werden kann, dass der Fitnesstracker an diesen Tagen nicht getragen wurde. Als Grenze für die Aktivität legen wir 5 Minuten am Tag fest.
Wir speichern die Tabelle in einem neuen Dataframe. 

In [ ]:
# 0 Schritte löschen 
daily_activity_cleaned <- daily_activity[daily_activity$TotalSteps != 0, ] 

# Weniger als 5 Minuten Aktivität am Tag löschen 
daily_activity_cleaned <- daily_activity_cleaned[daily_activity_cleaned$SedentaryMinutes < 1435, ]

## Datumformat vereinheitlichen
In  sleep_day und weight_info ist auch eine Uhrzeit-Angabe im Datum, während in den anderen Data Frames diese Angabe nicht angegeben ist. 
Wir wollen das Datum in sleep_day zu ausschließlich Tagen ändern. 

In [ ]:
# Überprüfen des Datentyps
str(sleep_day$SleepDay)
str(weight_info$Date)

Es handelt sich um Character Types. Es könnte allgemein sinnvoll seinen einen Date Type daraus zu machen, aber für diese Analyse ist es ausreichend die Uhrzeitangaben lediglich abzutrennen und die Daten im Chracter Type zu belassen. 

In [ ]:
# Wir behalten lediglich die ersten 9 Zeichen, also die Datumsangabe
sleep_day$SleepDay = substr(sleep_day$SleepDay, 1, 9)
weight_info$Date = substr(weight_info$Date, 1, 9)

Die Daten sind nun für die Analyse bereit. 

# Datenanalyse
## Anzahl an IDs und Messungen
Jede Tabelle enthält IDs. Wir wollen sehen, ob in jeder Tabelle die gleiche Anzahl an IDs enthalten ist. Außerdem wollen wir sehen, wie viele Messungen in jeder Tabelle enthalten sind. 

In [ ]:
# Anzahl IDs
id_count <- c(daily_activity_cleaned = n_distinct(daily_activity_cleaned$Id)) %>%
  c(daily_calories = n_distinct(daily_calories$Id)) %>%
  c(daily_intensities = n_distinct(daily_intensities$Id)) %>%
  c(daily_steps = n_distinct(daily_steps$Id)) %>%
  c(heartrate_seconds = n_distinct(heartrate_seconds$Id)) %>%
  c(sleep_day = n_distinct(sleep_day$Id)) %>%
  c(weight_info = n_distinct(weight_info$Id))
id_count

# Anzahl Messungen
observation_count <- c(daily_activity_cleaned = nrow(daily_activity_cleaned)) %>%
  c(daily_activity = nrow(daily_activity)) %>% 
  c(daily_calories = nrow(daily_calories)) %>%
  c(daily_intensities = nrow(daily_intensities)) %>%
  c(daily_steps = nrow(daily_steps)) %>%
  c(heartrate_seconds = nrow(heartrate_seconds)) %>%
  c(sleep_day = nrow(sleep_day)) %>%
  c(weight_info = nrow(weight_info))
observation_count

**Beobachtung**
* Die Data Frames zu Kalorien, intensität und Schritten zeigen gleich viele IDs und auch Messungen auf. 
* Die weiteren Funktionen sind augenscheinlich Zusatzfunktionen, die nicht von jedem Nutzer genutzt werden.

Es kann angenommen werden, dass es sich bei der Messung von Aktivität, Kalorien, Intensität und Schritten um Kernfunktionen des Trackers handelt. 
Aus dem Verhältnis von IDs und Beobachtungen ist außerdem erkennbar, dass die Funktionen, die von wenigern Nutzern genutzt werden, von diesen Nutzern auch weniger konstant genutzt werden. 

**Auffällig:** Die Pulsmessfunktion wurde anscheinend lediglich von 14 Personen genutzt. 

## Zusammenfassungen verschiedener Auswertungen 

In [ ]:
# Schritte, Distanz und Kalorien
daily_activity_cleaned %>%  
  select(TotalSteps,
         TrackerDistance,
         Calories) %>%
  summary()

# Puls
heartrate_seconds %>%  
  select(Value) %>%
  summary()

# Intensität
daily_intensities %>%  
  select(SedentaryMinutes,
  LightlyActiveMinutes,
  FairlyActiveMinutes,
  VeryActiveMinutes) %>%
  summary()

# Distanz
daily_intensities %>%  
  select(SedentaryActiveDistance,
  LightActiveDistance,
  ModeratelyActiveDistance,
  VeryActiveDistance) %>%
  summary()

**Beobachtungen**
* Der Mittlewert der Schritte liegt bei etwa 8.000 Schritten am Tag. Das sind weniger als die häufig genannten 10.000 Schritte, die ein Mensch am Tag gehen sollte.
* Der Mittelwert der Inaktivität (SedentaryMinutes) liegt bei 991 Minuten (16:31h) am Tag. Das ist ein deutlich zu hoher Wert. 
* Es werden im Mittel nur wenige Minuten wirkliche Aktivität (FairlyActiveMinutes und VeryActiveMinues) am Tag erreicht. 

In [ ]:
# Kalorien nach Aktivitätslevel. Als Grenze wurden 15 Minuten genutzt
filter(daily_activity_cleaned, VeryActiveMinutes > 15) %>% 
  select(Calories) %>% 
  summary()

filter(daily_activity_cleaned, FairlyActiveMinutes > 15) %>% 
  select(Calories) %>% 
  summary()

filter(daily_activity_cleaned, LightlyActiveMinutes > 30) %>% 
  select(Calories) %>% 
  summary()

filter(daily_activity_cleaned, SedentaryMinutes != 0) %>% 
  select(Calories) %>% 
  summary()

**Beobachtung**

Wie zu erwarten steigt der Kalorienverbrauch im Mittel mit steigender Aktivität an. Es zeigt sich allerdings auch, dass der Mittelwert bei den FairlyActiveMinutes und VeryActiveMinutes beinahe identisch ist. 

In [ ]:
# Schlaf
sleep_day %>%  
  select(TotalSleepRecords,
  TotalMinutesAsleep,
  TotalTimeInBed) %>%
  summary()

# Gewicht
weight_info %>%  
  select(WeightKg,
  BMI) %>%
  summary()

**Beobachtungen**
* Die meisten Personen der Studie schlafen 1x am Tag für im Mittelwert etwas unter 7 Stunden. Dieser Wert sollte zur optimalen Gesundheit höher sein. 
* Zusätzlich sind keine großen Schwankungen im BMI zu sehen, was entweder dafür spricht, dass Fitness-Tracker Nutzer allgemein in einem Normalgewicht liegen, oder bei dieser kleinen Datenlage ein Sampling Bias vorliegt. 

# Visualisierungen
Wir wollen verschiedenste Graphen erstellen, um mögliche Korrelationen und Auffälligkeiten aufzudecken. 

## Schritte im Vergleich zum Aktivitätslevel

In [ ]:
# - steps taken and sedentary active minutes
ggplot(data=daily_activity_cleaned, aes(x=TotalSteps, y=SedentaryMinutes)) + geom_point() + labs(title ="Schritte und Inaktivität")

# - steps taken and lightly active minutes 
ggplot(data=filter(daily_activity_cleaned, LightlyActiveMinutes != 0), aes(x=TotalSteps, y=LightlyActiveMinutes)) + geom_point() + labs(title ="Schritte und leichte Aktivität")

# - steps taken and fairly active minutes 
ggplot(data=filter(daily_activity_cleaned, FairlyActiveMinutes != 0), aes(x=TotalSteps, y=FairlyActiveMinutes)) + geom_point() + labs(title ="Schritte und Aktivität")

# - steps taken and very active minutes 
ggplot(data=filter(daily_activity_cleaned, VeryActiveMinutes != 0), aes(x=TotalSteps, y=VeryActiveMinutes)) + geom_point() + labs(title ="Schritte und hohe Aktivität")

**Beobachtungen**
* Es gibt eine Verbindung zwischen den gemachten Schritten und der inaktiven Zeit (SedentaryMinutes), was logisch erscheint. Allerdings scheint es dabei graphisch auch zwei "Gruppen" zu geben. 
* Leichte Aktivität und gemachte Schritte zeigen eine klare Verbindung zueinander. 
* Bei aktiven und sehr aktiven Zeiten zeigt sich erneut keine Verbindung zu den gemachten Schritten. 

Aus den Graphen kann geschlossen werden, dass Gehen als leichte Aktivität eingestuft wird, weshalb sich eine klare Verbindung zur Schrittzahl zeigt. Schwerere Aktivitäten, die aber weniger Schritte beinhalten, sind augenscheinlich nötig, um in eine höhere Aktivitätsmessung zu fallen, weshalb es keine klare Verbindung zu der gemachten Schrittzahl mehr gibt. 

## Zeit im Bett zu Zeit am Schlafen 

In [ ]:
ggplot(data=sleep_day, aes(x=TotalMinutesAsleep, y=TotalTimeInBed)) + geom_point() + labs(title ="Zeit imt Bett und Zeit am Schlafen")

**Beobachtung**

Es gibt eine beinahe vollständig lineare Verbindung zwischen der Zeit im Bett und der Zeit am Schlafen. Ich habe an dieser Stelle mit mehr Ausreißern gerechnet. 

## Gemachte Schritte zu Kalorien

In [ ]:
ggplot(data=daily_activity_cleaned, aes(x=TotalSteps, y=Calories)) + geom_point() + geom_smooth() + labs(title ="Schritte und Kalorien")

**Beobachtung**

Es zeigt sich eine Korrelation zwischen gemachten Schritten und verbrannten Kalorien. 

## Schlag am Tag und verbrannte Kalorien

In [ ]:
# Es muss ein Identifier erstellt werden, um die Werte eindeutig zuzuordnen
# Identifier sleep_day erstellen. 
sleep_day$identifier <- paste(sleep_day$Id, sleep_day$SleepDay, sep = " ")

#Identifier Kalorien erstellen
daily_calories$identifier <- paste(daily_calories$Id, daily_calories$ActivityDay, sep = " ")

# Tabellen verbinden
sleep_calories <- merge(sleep_day, select(daily_calories, -Id), by = "identifier") 

# ausschließlich die relevanten Spalten behalten
sleep_calories <- select(sleep_calories, -identifier, -TotalSleepRecords, -ActivityDay)

ggplot(data=sleep_calories, aes(x=TotalMinutesAsleep, y=Calories)) + geom_point() + labs(title ="Kalorien und Schlaf")

**Beobachtung**

Es scheint keine direkte Verbindung zwischen Kalorien und Schlafenszeit zu geben. Es zeigt sicher aber, dass Messungen, die einen höheren Kalorienverbrauch haben, eher im Bereich eines "gesunden" Schlafes von 7-8 Stunden fallen. 

## Schlaf und Gewicht

In [ ]:
# Es muss ein Identifier erstellt werden, um die Werte eindeutig zuzuordnen
# Identifier Schlaf erstellen 
sleep_day$identifier <- paste(sleep_day$Id, sleep_day$SleepDay, sep = " ")

#Identifier Gewicht erstellen
weight_info$identifier <- paste(weight_info$Id, weight_info$Date, sep = " ")

# Tabellen verbinden
sleep_weight <- merge(sleep_day, select(weight_info, -Id), by = "identifier") 

# ausschließlich die relevanten Spalten behalten
sleep_weight <- select(sleep_weight, -identifier, -TotalSleepRecords, -WeightPounds)

# Graph erstellen
ggplot(data=sleep_weight, aes(x=TotalMinutesAsleep, y=WeightKg)) + geom_point() + labs(title ="Gewicht und Schlaf")

**Beobachtung**

Es zeigt sich keine Korrelation zwischen Schlaf und Gewicht. Allerdings sieht man in diesem Graph eindeutig, dass ein Sampling Bias in den Daten enthalten ist. Das wird in einem Histogramm ebenfalls deutlich.  

## Überprüfen des Sampling Bias anhand des Gewichts der Teilnehmer


In [ ]:
ggplot(weight_info, aes(x=WeightKg)) + geom_histogram(binwidth = 0.5) + labs(title ="Gewichtsverteilung")

**Beobachtung**

Der Großteil der teilnehmenden Personen hat in etwa dasselbe Gewicht. Somit sind die Daten nicht repräsentativ für die Gesamtbevölkerung und wahrscheinlich auch nicht für die Gesamtheit aller Nutzer von Fitnesstrackern. Es liegt ein Sampling Bias bei den Daten vor. 

## Verbrachte Zeit in verschiedenen Aktivitätsleveln

In [ ]:
ggplot(daily_activity_cleaned, aes(x=SedentaryMinutes)) + geom_histogram(binwidth = 5) + labs(title ="Inaktivität")

ggplot(daily_activity_cleaned, aes(x=LightlyActiveMinutes)) + geom_histogram(binwidth = 5) + labs(title ="leichte Aktivität")

ggplot(daily_activity_cleaned, aes(x=FairlyActiveMinutes)) + geom_histogram(binwidth = 1) + labs(title ="Aktivität")

ggplot(daily_activity_cleaned, aes(x=VeryActiveMinutes)) + geom_histogram(binwidth = 1) + labs(title ="hohe Aktivität")

**Beobachtungen:**
* Bei den inaktiven Minuten gibt es augenscheinlich zwei "Spitzen", während es bei den anderen Aktivitätsleveln einen klaren Hochpunkt gibt. 
* Die aktiven und sehr aktiven Zeiten sind meist nur sehr kurz. 



# Limitierung der Daten 
Der Großteil der teilnehmenden Personen hat in etwa dasselbe Gewicht. Somit sind die Daten nicht repräsentativ für die Gesamtbevölkerung und wahrscheinlich auch nicht für die Gesamtheit aller Nutzer von Fitnesstrackern. Es liegt ein Sampling Bias bei den Daten vor.

# Zusammenfassung relevanter Beobachtungen 
Wir konnten verschiedene Beobachtungen und Auffälligkeiten in den Daten feststellen. Wenn wir annehmen, dass die Zielgruppe für einen Fitnesstracker aus Personen besteht, die allgemein an Gesundheit interessiert sind und wahrscheinlich ebenfalls daran interessiert sind ihr Gewicht in einem bestimmten Bereich zu halten (sprich im Zweifel mehr Kalorien zu verbrennen), erscheinen die folgenden Beobachtungen relevant: 

*  Der Mittlewert der Schritte liegt bei etwa 8.000 Schritten am Tag. Das sind weniger als die häufig genannten 10.000 Schritte, die ein Mensch am Tag gehen sollte.
* Der Mittelwert der Inaktivität (SedentaryMinutes) liegt bei 991 Minuten (16:31h) am Tag. Das ist ein deutlich zu hoher Wert. 
* Es werden im Mittel nur wenige Minuten wirkliche Aktivität (FairlyActiveMinutes und VeryActiveMinues) am Tag erreicht. 
* Der Kalorienverbrauch steigt im Mittel mit steigender Aktivität an. 
* Die meisten Personen der Studie schlafen 1x am Tag für im Mittelwert etwas unter 7 Stunden. Dieser Wert sollte zur optimalen Gesundheit höher sein. 
* Es zeigt sich eine Korrelation zwischen gemachten Schritten und verbrannten Kalorien. 

# Ideen für das Marketing 
Aus den oben genannten Beobachtungen lassen sich mehrere Punkte für das Marketing ableiten. Diese können gegliedert werden in technische Anpassungen und zusätzliche Produkte: 

**Technische Anpassungen** 
* Es könnten Erinnerungen zu Schritt-Erfolgen implementiert werden, um die Nutzer zu mehr Bewegung zu animieren. 
* Es sollte nach einer gewissen Zeit der Inaktivität, zum Beispiel 1-2 Stundne, eine Erinnerung erfolgen, die zu mehr Bewegung anregt. 
* Es könnte eine Erinnerung zur optimalen Schlafenszeit gesetzt werden, um für mehr Schlaf zu sorgen und damit zu einer Steigerung der Gesundheit beizutragen. 

**Zusätzliche Produkte / Upselling-Möglichkeit**

Die Daten zeigen, dass mit relativ wenig Zeit in höheren Aktivitätsleveln (im Mittel nur ein paar Minuten am Tag) deutlich mehr Kalorien verbrannt werden und auch eine Verbesserung der Schlafqualität (Zeit am Schlafen) damit einhergeht. Zusätzlich ist zu erkennen, dass die Nutzer einen ansonsten relativ inaktiven Lebensstil führen, beziehungsweise augenscheinlich wenig Zeit für lange Sporteinheiten haben. 

Aus diesen drei Gründen erscheint es sinnvoll zum weiteren Wachstum des Unternehmens ein zusätzliches Produkt anzubieten, möglicherweise eine monatliche Mitgliedschaft oder Ähnliches zubuchbar zum Fitnesstracker, in der ein HIIT-Trainingsprogramm (15 Minuten an ein paar Tagen der Woche) angeboten wird, welches regelmäßig aktualisiert wird. Die genannten Punkte (mehr Kalorienverbrauch, besserer Schlaf, Zeitknappheit) können dabiedabei als Verkaufsargumente im Marketing genutzt werden. 

# Weitere Schritte 
Als nächstes wird es nötig sein weitere Datenquellen herbeizuziehen, da es sich bisher um eine relativ kleine Datenquelle handelt und ein Bias in den Daten festgestellt wurde. Sollte sich bei größeren Datensätzen ein ähnliches Bild zeigen, könnten im nächsten Schritt aktuelle Kunden befragt werden, ob diese an etwaigen Fitnessprogrammen interessiert wären. 
